# Breast Cancer Detection using PyTorch (Logistic Regression)

This notebook implements a **Binary Classification Model** using **Logistic Regression** built from scratch with **PyTorch**.

### Complete Pipeline:
1. **Import Libraries**: Load necessary Python libraries.
2. **Load Dataset**: Fetch the Breast Cancer dataset from a remote URL.
3. **Data Cleaning**: Remove unused columns.
4. **Train-Test Split**: Divide data into training and testing sets.
5. **Feature Scaling**: Normalize features using StandardScaler.
6. **Label Encoding**: Convert categorical labels ('M', 'B') into numerical values (1, 0).
7. **Convert to PyTorch Tensors**: Prepare data for PyTorch.
8. **Define Neural Network Model**: Create a Logistic Regression model class.
9. **Define Loss Function**: Implement Binary Cross Entropy Loss manually.
10. **Training Loop**: Train the model using Gradient Descent.
11. **Evaluation**: Test the model's accuracy on unseen data.

## 1️⃣ Import Required Libraries

We will import:
- `numpy` and `pandas` for data manipulation.
- `torch` for building the neural network.
- `sklearn` for data preprocessing and splitting.

In [1]:
import numpy as np
import pandas as pd
import torch

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

print("Libraries imported successfully!")

Libraries imported successfully!


## 2️⃣ Load Dataset

We load the dataset directly from the raw GitHub URL.
The dataset contains features computed from a digitized image of a fine needle aspirate (FNA) of a breast mass.

In [3]:
url = "https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv"
df = pd.read_csv(url)

print("Dataset Shape:", df.shape)
df.head()

Dataset Shape: (569, 33)


,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


## 3️⃣ Data Cleaning

We inspect the dataset and remove columns that are not useful for training, specifically:
- `id`: Unique identifier for each sample.
- `Unnamed: 32`: An empty column often created during CSV reading due to trailing commas.

In [4]:
# Remove unnecessary columns
df.drop(columns=['id', 'Unnamed: 32'], inplace=True)

print("Shape after dropping columns:", df.shape)

Shape after dropping columns: (569, 31)


## 4️⃣ Train-Test Split

We separate the features (`X`) from the target variable (`y`).
- `X`: All columns except diagnosis.
- `y`: The diagnosis column ('M' or 'B').

Then, we split the data into training (80%) and testing (20%) sets.

In [5]:
X = df.iloc[:, 1:]      # All features (excluding diagnosis at index 0)
y = df.iloc[:, 0]       # diagnosis column

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42
)

print("Training Features Shape:", X_train.shape)
print("Testing Features Shape:", X_test.shape)

Training Features Shape: (455, 30)
Testing Features Shape: (114, 30)


## 5️⃣ Feature Scaling

Standardization calculates the mean and standard deviation for each feature and scales the data so that the mean is 0 and standard deviation is 1.
This is crucial for neural networks to converge faster.

In [6]:
scaler = StandardScaler()

# Fit on training set only to avoid data leakage
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print("First 3 rows of Scaled Training Data:\n", X_train[:3])

First 3 rows of Scaled Training Data:
 [[-1.44075296 -0.43531947 -1.36208497 -1.1391179   0.78057331  0.71892128
   2.82313451 -0.11914956  1.09266219  2.45817261 -0.26380039 -0.01605246
  -0.47041357 -0.47476088  0.83836493  3.25102691  8.43893667  3.39198733
   2.62116574  2.06120787 -1.23286131 -0.47630949 -1.24792009 -0.97396758
   0.72289445  1.18673232  4.67282796  0.9320124   2.09724217  1.88645014]
 [ 1.97409619  1.73302577  2.09167167  1.85197292  1.319843    3.42627493
   2.01311199  2.66503199  2.1270036   1.55839569  0.80531919 -0.81268678
   0.75195659  0.87716951 -0.89605315  1.18122247  0.18362761  0.60059598
  -0.31771686  0.52963649  2.17331385  1.3112795   2.08161691  2.1374055
   0.76192793  3.26560084  1.92862053  2.6989469   1.89116053  2.49783848]
 [-1.39998202 -1.24962228 -1.34520926 -1.10978518 -1.33264483 -0.30735463
  -0.36555756 -0.69650228  1.93033305  0.95437877  0.02752055  1.96305996
  -0.12095781 -0.35077918  0.57276579  0.7394992   0.32065553  0.5894622

## 6️⃣ Label Encoding

The target variable has categorical values:
- **M** (Malignant) -> Encoded as **1**
- **B** (Benign) -> Encoded as **0**

In [7]:
encoder = LabelEncoder()

y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

print("Encoded Labels Example (First 10):", y_train[:10])

Encoded Labels Example (First 10): [0 1 0 0 0 1 0 0 0 1]


## 7️⃣ Convert to PyTorch Tensors

PyTorch works with Tensors. We convert our numpy arrays to PyTorch tensors and ensure they are of type `float`.
We also reshape the target labels to be column vectors `(N, 1)`.

In [8]:
X_train_tensor = torch.from_numpy(X_train).float()
X_test_tensor  = torch.from_numpy(X_test).float()

# Reshape labels to (N, 1)
y_train_tensor = torch.from_numpy(y_train).float().view(-1, 1)
y_test_tensor  = torch.from_numpy(y_test).float().view(-1, 1)

print("X_train_tensor shape:", X_train_tensor.shape)
print("y_train_tensor shape:", y_train_tensor.shape)

X_train_tensor shape: torch.Size([455, 30])
y_train_tensor shape: torch.Size([455, 1])


## 8️⃣ Define Neural Network Model

We define a simple Logistic Regression model class.
- **Attributes**:
    - `weights`: Initialized randomly.
    - `bias`: Initialized to zero.
- **Methods**:
    - `forward(X)`: Computes the linear combination `z = Xw + b` followed by the Sigmoid activation.
    - `loss_function(y_pred, y_true)`: Computes the Binary Cross Entropy loss.

In [9]:
class MySimpleNN:

    def __init__(self, X):
        # Initialize weights randomly
        # X.shape[1] is the number of features (30 in this case)
        self.weights = torch.randn(X.shape[1], 1, requires_grad=True)

        # Initialize bias
        self.bias = torch.zeros(1, requires_grad=True)

    def forward(self, X):
        # Linear transformation: z = X * w + b
        z = torch.matmul(X, self.weights) + self.bias

        # Sigmoid activation for Binary Classification
        # output range: [0, 1]
        y_pred = torch.sigmoid(z)

        return y_pred

    def loss_function(self, y_pred, y_true):
        """
        Binary Cross Entropy Loss (Manual implementation)
        Loss = - mean( y_true * log(y_pred) + (1 - y_true) * log(1 - y_pred) )
        """
        # Small epsilon to avoid log(0)
        epsilon = 1e-7
        y_pred = torch.clamp(y_pred, epsilon, 1 - epsilon)

        loss = -(
            y_true * torch.log(y_pred) +
            (1 - y_true) * torch.log(1 - y_pred)
        ).mean()

        return loss

## 9️⃣ & 🔟 Training Loop

We train the model over a fixed number of epochs.
In each epoch, we perform:
1. **Forward Pass**: Compute predictions.
2. **Loss Calculation**: Measure how far off predictions are from truth.
3. **Backward Pass**: Compute gradients.
4. **Parameter Update**: Adjust weights and bias using Gradient Descent.

Hyperparameters:
- `learning_rate`: 0.1
- `epochs`: 25

In [10]:
# Hyperparameters
learning_rate = 0.1
epochs = 25

# Create model instance
model = MySimpleNN(X_train_tensor)

print("Starting Training...\n")

for epoch in range(epochs):

    # 1. Forward pass
    y_pred = model.forward(X_train_tensor)

    # 2. Calculate loss
    loss = model.loss_function(y_pred, y_train_tensor)

    # 3. Backpropagation (Calculate gradients)
    loss.backward()

    # 4. Update parameters (Gradient Descent)
    with torch.no_grad():
        model.weights -= learning_rate * model.weights.grad
        model.bias    -= learning_rate * model.bias.grad

        # Reset gradients for the next iteration
        model.weights.grad.zero_()
        model.bias.grad.zero_()

    print(f"Epoch [{epoch+1}/{epochs}] Loss: {loss.item():.4f}")

Starting Training...

Epoch [1/25] Loss: 1.3787
Epoch [2/25] Loss: 1.2878
Epoch [3/25] Loss: 1.2045
Epoch [4/25] Loss: 1.1300
Epoch [5/25] Loss: 1.0633
Epoch [6/25] Loss: 1.0035
Epoch [7/25] Loss: 0.9496
Epoch [8/25] Loss: 0.9009
Epoch [9/25] Loss: 0.8566
Epoch [10/25] Loss: 0.8162
Epoch [11/25] Loss: 0.7792
Epoch [12/25] Loss: 0.7451
Epoch [13/25] Loss: 0.7126
Epoch [14/25] Loss: 0.6810
Epoch [15/25] Loss: 0.6518
Epoch [16/25] Loss: 0.6247
Epoch [17/25] Loss: 0.5996
Epoch [18/25] Loss: 0.5763
Epoch [19/25] Loss: 0.5545
Epoch [20/25] Loss: 0.5342
Epoch [21/25] Loss: 0.5152
Epoch [22/25] Loss: 0.4974
Epoch [23/25] Loss: 0.4807
Epoch [24/25] Loss: 0.4649
Epoch [25/25] Loss: 0.4501


## 1️⃣1️⃣ Model Evaluation

We evaluate the trained model on the unseen Test data.
- Compute predictions on `X_test_tensor`.
- Threshold probabilities at 0.5 to convert to binary classes (0 or 1).
- Compare predicted classes with true labels (`y_test_tensor`) to calculate accuracy.

In [11]:
with torch.no_grad():

    # Get probabilities for test set
    y_pred = model.forward(X_test_tensor)

    # Convert probabilities to 0/1 classes
    y_pred_cls = (y_pred > 0.5).float()

    # Calculate accuracy
    accuracy = (y_pred_cls == y_test_tensor).float().mean()

    print(f"Test Accuracy: {accuracy.item():.4f}")
    print(f"Test Accuracy Score: {accuracy.item() * 100:.2f}%")

Test Accuracy: 0.9123
Test Accuracy Score: 91.23%
